In [9]:
import pandas as pd

In [15]:
#Testing with 2010 Bronx data
f = open('2010/2010_bronx.csv','r')
df = pd.read_csv(f)
f.close()

In [20]:
df.count()

Unnamed: 0                        4755
BOROUGH                           4755
NEIGHBORHOOD                      4755
BUILDING CLASS CATEGORY           4755
TAX CLASS AT PRESENT              4755
BLOCK                             4755
LOT                               4755
EASE-MENT                         4755
BUILDING CLASS AT PRESENT         4755
ADDRESS                           4755
APARTMENT NUMBER                  4755
ZIP CODE                          4755
RESIDENTIAL UNITS                 4755
COMMERCIAL UNITS                  4755
TOTAL UNITS                       4755
LAND SQUARE FEET                  4755
GROSS SQUARE FEET                 4755
YEAR BUILT                        4755
TAX CLASS AT TIME OF SALE         4755
BUILDING CLASS AT TIME OF SALE    4755
SALE PRICE                        4755
SALE DATE                         4755
dtype: int64

In [37]:
df2 = df[df['SALE PRICE'] > 1000]

In [38]:
#columns to keep
to_keep =(
[u'BOROUGH', u'NEIGHBORHOOD', u'BUILDING CLASS CATEGORY',
 u'TAX CLASS AT PRESENT', u'BLOCK', u'LOT',
       u'BUILDING CLASS AT PRESENT', u'ADDRESS',
       u'ZIP CODE', u'RESIDENTIAL UNITS', u'COMMERCIAL UNITS', u'TOTAL UNITS',
       u'LAND SQUARE FEET', u'GROSS SQUARE FEET', u'YEAR BUILT',
       u'TAX CLASS AT TIME OF SALE', u'BUILDING CLASS AT TIME OF SALE',
       u'SALE PRICE'])

In [40]:
df2[to_keep].ind

BOROUGH                           2911
NEIGHBORHOOD                      2911
BUILDING CLASS CATEGORY           2911
TAX CLASS AT PRESENT              2911
BLOCK                             2911
LOT                               2911
BUILDING CLASS AT PRESENT         2911
ADDRESS                           2911
ZIP CODE                          2911
RESIDENTIAL UNITS                 2911
COMMERCIAL UNITS                  2911
TOTAL UNITS                       2911
LAND SQUARE FEET                  2911
GROSS SQUARE FEET                 2911
YEAR BUILT                        2911
TAX CLASS AT TIME OF SALE         2911
BUILDING CLASS AT TIME OF SALE    2911
SALE PRICE                        2911
dtype: int64